In [1]:
import datetime
import pandas as pd
import numpy as np



In [2]:
from alpha_vantage.timeseries import TimeSeries
alphavantage_api_key = 'LHSF13XCYF6MXI3J'
ts = TimeSeries(key= alphavantage_api_key, output_format='pandas')



## Function: Get Stock Price from AlphaVantage

In [34]:
data, meta_data = ts.get_daily(symbol='USO', outputsize='compact')
data = data.reset_index()
price = data.loc[data.date == '2017-01-10','4. close']

In [42]:
def get_price(ticker,date):
    try:
        data, meta_data = ts.get_daily(symbol=ticker, outputsize='compact')
        data = data.reset_index()
        price = data.loc[data.date == date,'4. close']
        return price[0]
    except:
        pass


In [43]:
csv_path = "Oil Company Tweet Data_date_mod.csv"
df = pd.read_csv(csv_path, encoding='ISO-8859-1')

df.head()

,Unnamed: 0,Date,Media Sources,Negative,Neutral,Positive,Source Account,Tweet Polarity,Tweets Ago
0,0,2017-01-10,@OPECnews,0.000,1.000,0.000,GasBuddy,0.0000,1
1,270,2017-10-07,@Shell,0.232,0.674,0.094,Richard Brooks,-0.5719,91


In [45]:
df['Media Sources'].unique()

array(['@OPECnews', '@Shell'], dtype=object)

## Map Media Source to Ticker to later Retrieve Stock Price

In [46]:
oil_ticker = {'@Shell': 'AAPL',
             '@OPECnews': 'USO',
             '@exxonmobil':'XOM'}

## Get Stock Ticker in order to retrieve Price

In [47]:
df['Ticker'] = [oil_ticker[x] for x in df['Media Sources']]

In [49]:
df.head()

,Unnamed: 0,Date,Media Sources,Negative,Neutral,Positive,Source Account,Tweet Polarity,Tweets Ago,Ticker
0,0,2017-01-10,@OPECnews,0.000,1.000,0.000,GasBuddy,0.0000,1,USO
1,270,2017-10-07,@Shell,0.232,0.674,0.094,Richard Brooks,-0.5719,91,AAPL


## Get Stock Price (Ticker, Date)

In [21]:
df['Price'] = df.apply(lambda x: get_price(x['Ticker'],x['Date']), axis=1)
df.head()

,Unnamed: 0,Date,Media Sources,Negative,Neutral,Positive,Source Account,Tweet Polarity,Tweets Ago,Ticker,Price
0,0,2017-01-10,@OPECnews,0.000,1.000,0.000,GasBuddy,0.0000,1,USO,None
1,270,2017-10-07,@Shell,0.232,0.674,0.094,Richard Brooks,-0.5719,91,AAPL,None
